In [ ]:
import sys, copy

###########################
# A) Total Square Calculation
###########################
def total_filled_squares_grid(row_constraints, col_constraints):
    total_rows = sum(sum(row) for row in row_constraints)
    total_cols = sum(sum(col) for col in col_constraints)
    if total_rows != total_cols:
        raise ValueError("Row and column totals do not match!")
    return total_rows

###########################
# B) Global Confidence Filling (Step 1)
###########################
def confidence_fill_line(n, constraints):
    # Standard confidence fill over the full line
    if not constraints:
        return [0] * n
    S = sum(constraints) + (len(constraints) - 1)
    f = n - S
    result = [-1] * n
    if S == n:
        idx = 0
        for block in constraints:
            for _ in range(block):
                result[idx] = 1
                idx += 1
            if idx < n:
                result[idx] = 0
                idx += 1
        return result
    P = 0
    for i, block in enumerate(constraints):
        if i == 0:
            P += block
        else:
            P += block + 1
        if block > f:
            fill_count = block - f
            for k in range(fill_count):
                result[P - 1 - k] = 1
    return result

###########################
# C-1) Range Calculation & Allocation (Basic)
###########################
def calculate_ranges_and_allocation(n, constraints):
    k = len(constraints)
    if k == 0:
        return [], {}, set()
    l_values = []
    for i in range(k):
        li = 1 if i == 0 else l_values[i-1] + constraints[i-1] + 1
        l_values.append(li)
    r_values = []
    for i in range(k):
        future_sum = sum(constraints[i+1:]) if i+1 < k else 0
        gaps = (k - i - 1)
        ri = n - future_sum - gaps
        r_values.append(ri)
    block_ranges = list(zip(l_values, r_values))
    square_alloc = {}
    for pos in range(1, n+1):
        square_alloc[pos] = []
        for i, (l, r) in enumerate(block_ranges):
            if l <= pos <= r:
                square_alloc[pos].append(i)
    region_allocation = {i: set() for i in range(k)}
    overlap_allocation = set()
    for pos, blocks in square_alloc.items():
        if len(blocks) == 1:
            region_allocation[blocks[0]].add(pos)
        elif len(blocks) > 1:
            overlap_allocation.add(pos)
    return block_ranges, region_allocation, overlap_allocation

###########################
# C-2) Basic Step 2: Deduction (Range Process, Corner Attack, Fill Between)
###########################
def step2_process_line(row, constraints):
    n = len(row)
    block_ranges, region_alloc, _ = calculate_ranges_and_allocation(n, constraints)
    k = len(constraints)
    mapped = {i: set() for i in range(k)}
    for pos in range(1, n+1):
        if row[pos-1] == 1:
            for i in range(k):
                if pos in region_alloc[i]:
                    mapped[i].add(pos)
    pos = 1
    while pos <= n:
        if row[pos-1] == 1:
            start = pos
            while pos <= n and row[pos-1] == 1:
                pos += 1
            end = pos - 1
            for i in range(k):
                if any(s in region_alloc[i] for s in range(start, end+1)):
                    for s in range(start, end+1):
                        mapped[i].add(s)
        else:
            pos += 1
    new_row = row[:]
    for i, (l, r) in enumerate(block_ranges):
        a = constraints[i]
        left_corner = set(range(l, l + a))
        right_corner = set(range(r - a + 1, r + 1))
        for s in sorted(mapped[i]):
            if s in left_corner:
                for pos in range(s, l + a):
                    if l <= pos <= r:
                        new_row[pos-1] = 1
            if s in right_corner:
                for pos in range(r - a + 1, s+1):
                    if l <= pos <= r:
                        new_row[pos-1] = 1
    for i in range(k):
        if mapped[i]:
            sorted_positions = sorted(mapped[i])
            for j in range(len(sorted_positions) - 1):
                for pos in range(sorted_positions[j], sorted_positions[j+1] + 1):
                    new_row[pos-1] = 1
    return new_row

###########################
# D) Basic Cross Filling (Step 3)
###########################
def fill_crosses_step3(row, constraints):
    n = len(row)
    k = len(constraints)
    new_row = row[:]
    segments = []
    i = 0
    while i < n:
        if new_row[i] == 1:
            start = i
            while i < n and new_row[i] == 1:
                i += 1
            segments.append((start, i-1))
        else:
            i += 1
    if len(segments) == k:
        all_match = True
        for (s, e), a in zip(segments, constraints):
            if (e - s + 1) != a:
                all_match = False
                break
        if all_match:
            for i in range(n):
                if new_row[i] == -1:
                    new_row[i] = 0
    block_ranges, region_alloc, _ = calculate_ranges_and_allocation(n, constraints)
    print(region_alloc)
    for i in range(k):
        sz=len(region_alloc[i])
        if(sz>=1):
          li, ri = list(region_alloc[i])[0], list(region_alloc[i])[sz-1]
          a = constraints[i]
          for p in region_alloc[i]:
              if new_row[p-1] == 1:
                  left_limit = p - a
                  if left_limit >= li:
                      for pos in range(li, left_limit + 1):
                          if new_row[pos-1] == -1:
                              new_row[pos-1] = 0
                  right_limit = p + a
                  if right_limit <= ri:
                      for pos in range(right_limit, ri + 1):
                          if new_row[pos-1] == -1:
                              new_row[pos-1] = 0
    for i in range(k):
        li, ri = block_ranges[i]
        a = constraints[i]
        filled_positions = [p for p in range(li, ri+1) if new_row[p-1] == 1]
        if not filled_positions:
            continue
        segs = []
        seg_start = filled_positions[0]
        seg_end = filled_positions[0]
        for pos in filled_positions[1:]:
            if pos == seg_end + 1:
                seg_end = pos
            else:
                segs.append((seg_start, seg_end))
                seg_start = pos
                seg_end = pos
        segs.append((seg_start, seg_end))
        for seg in segs:
            if seg[1] - seg[0] + 1 == a:
                if seg[0] > li and new_row[seg[0]-2] == -1:
                    new_row[seg[0]-2] = 0
                if seg[1] < ri and new_row[seg[1]] == -1:
                    new_row[seg[1]] = 0
    return new_row

###########################
# E) Extra: Range Updation & Local Confidence Filling on Updated Ranges
###########################
def update_block_ranges_line(row, constraints):
    n = len(row)
    orig_ranges, region_alloc, _ = calculate_ranges_and_allocation(n, constraints)
    k = len(constraints)
    updated_ranges = []
    for i in range(k):
        l, r = orig_ranges[i]
        new_l, new_r = l, r
        for x in range(l, r+1):
            if row[x-1] != -1 and x not in region_alloc[i]:
                interfering = None
                for j in range(k):
                    if j != i and x in region_alloc[j]:
                        interfering = j
                        break
                if interfering is not None:
                    if interfering > i:
                        new_r = min(new_r, x - 1)
                    elif interfering < i:
                        new_l = max(new_l, x + 1)
                else:
                    if row[x-1] == 0:
                        mid = (l + r) // 2
                        if x <= mid:
                            new_r = min(new_r, x - 1)
                        else:
                            new_l = max(new_l, x + 1)
        mapped = {pos for pos in range(l, r+1) if row[pos-1] == 1 and pos in region_alloc[i]}
        if mapped:
            x_min = min(mapped)
            x_max = max(mapped)
            frt2=0
            for y in range(new_l, new_r+1):
                if row[y-1] == 0:
                    frt2=1
                    if y < x_min:
                        new_l = max(new_l, y + 1)
                    if y > x_max:
                        new_r = min(new_r, y - 1)
                if(frt2==0):
                  new_l=max(new_l,x_max-constraints[i]+1)
                  new_r=min(new_r, x_min+constraints[i]-1)
        frt=0
        for sanandh in range(new_l,new_r+1):
          if(row[sanandh-1]==1):
             frt=1
        frt2=0
        poch=-1
        pochtype=-1
        if(frt==0):
          for sanandh in range(new_l,new_r):
            if(row[sanandh-1]!=row[sanandh] and row[sanandh]==0):
              poch=sanandh
              pochtype=0
              frt2+=1
            elif(row[sanandh-1]!=row[sanandh] and row[sanandh]==-1):
              poch=sanandh
              pochtype=1
              frt2+=1
        if(frt2==1 and pochtype==1):
          new_l=sanandh
        elif(frt2==1 and pochtype==0):
          new_r=sanandh-1

        updated_ranges.append((new_l, new_r))

    square_alloc = {}
    for pos in range(1, n+1):
        square_alloc[pos] = []
        for i, (l, r) in enumerate(updated_ranges):
            if l <= pos <= r:
                square_alloc[pos].append(i)
    region_allocation = {i: set() for i in range(k)}
    overlap_allocation = set()
    for pos, blocks in square_alloc.items():
        if len(blocks) == 1:
            region_allocation[blocks[0]].add(pos)
        elif len(blocks) > 1:
            overlap_allocation.add(pos)

    return updated_ranges, region_allocation, overlap_allocation

def confidence_fill_line_with_updated_ranges(row, constraints, upd_ranges):
    new_row = row[:]
    for i, a in enumerate(constraints):
        L, R = upd_ranges[i]
        # Compute overlapping region between leftmost placement and rightmost placement
        start_conf = max(L, R - a + 1)
        end_conf = min(L + a - 1, R)
        for pos in range(start_conf, end_conf + 1):
            if new_row[pos-1] == -1:
                new_row[pos-1] = 1
    return new_row

def step2_process_line_with_ranges(row, constraints, upd_ranges):
    n = len(row)
    k = len(constraints)
    square_alloc = {}
    for pos in range(1, n+1):
        square_alloc[pos] = []
        for i, (l, r) in enumerate(upd_ranges):
            if l <= pos <= r:
                square_alloc[pos].append(i)
    region_allocation = {i: set() for i in range(k)}
    overlap_allocation = set()
    for pos, blocks in square_alloc.items():
        if len(blocks) == 1:
            region_allocation[blocks[0]].add(pos)
        elif len(blocks) > 1:
            overlap_allocation.add(pos)
    region_alloc = region_allocation
    mapped = {i: set() for i in range(k)}
    for pos in range(1, n+1):
        if row[pos-1] == 1:
            for i in range(k):
                if pos in region_alloc[i]:
                    mapped[i].add(pos)
    new_row = row[:]
    for i in range(k):
        L, R = upd_ranges[i]
        a = constraints[i]
        left_corner = set(range(L, L + a))
        right_corner = set(range(R - a + 1, R + 1))
        for s in sorted(mapped[i]):
            if s in left_corner:
                for pos in range(s, L + a):
                    if L <= pos <= R:
                        new_row[pos-1] = 1
            if s in right_corner:
                for pos in range(R - a + 1, s+1):
                    if L <= pos <= R:
                        new_row[pos-1] = 1
    for i in range(k):
        if mapped[i]:
            sorted_positions = sorted(mapped[i])
            for j in range(len(sorted_positions)-1):
                for pos in range(sorted_positions[j], sorted_positions[j+1] + 1):
                    new_row[pos-1] = 1
    return new_row

def fill_crosses_step3_with_ranges(row, constraints, upd_ranges, upd_region_alloc):
    n = len(row)
    new_row = row[:]
    k = len(constraints)
    segments = []
    i = 0
    while i < n:
        if new_row[i] == 1:
            start = i
            while i < n and new_row[i] == 1:
                i += 1
            segments.append((start, i-1))
        else:
            i += 1
    if len(segments) == k:
        all_match = True
        for (s, e), a in zip(segments, constraints):
            if (e - s + 1) != a:
                all_match = False
                break
        if all_match:
            for i in range(n):
                if new_row[i] == -1:
                    new_row[i] = 0
    for i in range(k):
        sz=len(upd_region_alloc[i])
        if(sz>=1):
          L, R = list(upd_region_alloc[i])[0], list(upd_region_alloc[i])[sz-1]
          a = constraints[i]
          for pos in range(L, R+1):
              if new_row[pos-1] == 1:
                  left_limit = pos - a
                  if left_limit >= L:
                      for p in range(L, left_limit+1):
                          if new_row[p-1] == -1:
                              new_row[p-1] = 0
                  right_limit = pos + a
                  if right_limit <= R:
                      for p in range(right_limit, R+1):
                          if new_row[p-1] == -1:
                              new_row[p-1] = 0
    for i in range(k):
        L, R = upd_ranges[i]
        a = constraints[i]
        filled_positions = [p for p in range(L, R+1) if new_row[p-1] == 1]
        if not filled_positions:
            continue
        segs = []
        seg_start = filled_positions[0]
        seg_end = filled_positions[0]
        for pos in filled_positions[1:]:
            if pos == seg_end + 1:
                seg_end = pos
            else:
                segs.append((seg_start, seg_end))
                seg_start = pos
                seg_end = pos
        segs.append((seg_start, seg_end))
        for seg in segs:
            if seg[1] - seg[0] + 1 == a:
                if seg[0] > L and new_row[seg[0]-2] == -1:
                    new_row[seg[0]-2] = 0
                if seg[1] < R and new_row[seg[1]] == -1:
                    new_row[seg[1]] = 0
    return new_row

##########################
# Grid and Helper Functions
##########################
def extract_column(grid, j):
    return [row[j] for row in grid]

def update_column(grid, j, col):
    for i in range(len(grid)):
        grid[i][j] = col[i]

def print_grid(grid):
    for row in grid:
        print(" ".join(str(x) for x in row))

##########################
# Main Solver Procedure
##########################
def solve_nonogram(row_constraints, col_constraints):
    n = len(row_constraints)  # grid is n x n
    total = total_filled_squares_grid(row_constraints, col_constraints)
    print("Total filled squares (expected):", total)

    # Initialize grid with -1 (undecided)
    grid = [[-1 for _ in range(n)] for _ in range(n)]

    # Step 1: Global Confidence Filling for rows and columns.
    for i in range(n):
        new_row = confidence_fill_line(n, row_constraints[i])
        for j in range(n):
            if new_row[j] != -1:
                grid[i][j] = new_row[j]
    for j in range(n):
        new_col = confidence_fill_line(n, col_constraints[j])
        for i in range(n):
            if new_col[i] != -1:
                grid[i][j] = new_col[i]
    print("\nAfter Step 1 (Global Confidence Filling) - Grid:")
    print_grid(grid)

    # Global iterative loop until grid converges.
    global_changed = True
    global_iter = 0
    max_global_iter = 50  # limit for our 5x5 puzzle
    while global_changed and global_iter < max_global_iter:
        global_iter += 1
        prev_grid = copy.deepcopy(grid)

        # Basic Step 2: Deduction.
        changed = True
        iter2_basic = 0
        while changed:
            changed = False
            iter2_basic += 1
            for i in range(n):
                new_row = step2_process_line(grid[i], row_constraints[i])
                if new_row != grid[i]:
                    grid[i] = new_row
                    changed = True
            for j in range(n):
                col = extract_column(grid, j)
                new_col = step2_process_line(col, col_constraints[j])
                if new_col != col:
                    update_column(grid, j, new_col)
                    changed = True
            if changed:
                print(f"\nAfter Basic Step 2, iter {iter2_basic} (global iter {global_iter}) - Grid:")
                print_grid(grid)

        # Basic Step 3: Cross Filling.
        for i in range(n):
            grid[i] = fill_crosses_step3(grid[i], row_constraints[i])
        for j in range(n):
            col = extract_column(grid, j)
            new_col = fill_crosses_step3(col, col_constraints[j])
            update_column(grid, j, new_col)
        print(f"\nAfter Basic Step 3 (global iter {global_iter}) - Grid:")
        print_grid(grid)

        # Extra iterative loop: Range Updation and Local Confidence Fill.
        range_changed = True
        range_iter = 0
        while range_changed:
            range_changed = False
            range_iter += 1
            for i in range(n):
                upd_ranges, upd_region_alloc, _ = update_block_ranges_line(grid[i], row_constraints[i])
                print(upd_ranges)
                new_row = confidence_fill_line_with_updated_ranges(grid[i], row_constraints[i], upd_ranges)
                print(new_row)
                new_row = step2_process_line_with_ranges(new_row, row_constraints[i], upd_ranges)
                print(new_row)
                new_row = fill_crosses_step3_with_ranges(new_row, row_constraints[i], upd_ranges, upd_region_alloc)
                print(new_row)
                if new_row != grid[i]:
                    grid[i] = new_row
                    range_changed = True
            for j in range(n):
                col = extract_column(grid, j)
                upd_ranges, upd_region_alloc, _ = update_block_ranges_line(col, col_constraints[j])
                print(upd_ranges)
                new_col = confidence_fill_line_with_updated_ranges(col, col_constraints[j], upd_ranges)
                print(new_col)
                new_col = step2_process_line_with_ranges(new_col, col_constraints[j], upd_ranges)
                print(new_col)
                new_col = fill_crosses_step3_with_ranges(new_col, col_constraints[j], upd_ranges, upd_region_alloc)
                print(new_col)
                if new_col != col:
                    update_column(grid, j, new_col)
                    range_changed = True
            if range_changed:
                print(f"\nAfter Range Updation & Local Confidence, iter {range_iter} (global iter {global_iter}) - Grid:")
                print_grid(grid)

        if grid != prev_grid:
            global_changed = True
        else:
            global_changed = False
        print(f"\nEnd of Global Iteration {global_iter} - Grid:")
        print_grid(grid)

    print("\nFinal Grid State:")
    print_grid(grid)
    return grid

##########################
# Main: Input & Solve for 5x5 Puzzle
##########################
if __name__ == '__main__':
    try:
        n = int(input("Enter grid size (n x n): "))
        print("Enter row constraints (one row per line; comma-separated numbers, or empty for no block):")
        row_constraints = []
        for i in range(n):
            line = input(f"Row {i+1} constraints: ").strip()
            if line == "":
                row_constraints.append([])
            else:
                row_constraints.append([int(x.strip()) for x in line.split(",") if x.strip() != ""])
        print("Enter column constraints (one column per line; comma-separated numbers, or empty for no block):")
        col_constraints = []
        for j in range(n):
            line = input(f"Column {j+1} constraints: ").strip()
            if line == "":
                col_constraints.append([])
            else:
                col_constraints.append([int(x.strip()) for x in line.split(",") if x.strip() != ""])
    except Exception as e:
        print("Invalid input:", e)
        sys.exit(1)

    final_grid = solve_nonogram(row_constraints, col_constraints)


    # For our 5x5 puzzle, use:
    # Row constraints: [2], [1,1], [3], [3], [3]
    # Column constraints: [1], [2], [3], [4], [3]
    #row_constraints = [[2], [1,1], [3], [3], [3]]
    #col_constraints = [[1], [2], [3], [4], [3]]
    #print("Solving 5x5 Nonogram with:")
    #print("Row constraints:", row_constraints)
    #print("Column constraints:", col_constraints)
    #final_grid = solve_nonogram(row_constraints, col_constraints)

Enter grid size (n x n): 10
Enter row constraints (one row per line; comma-separated numbers, or empty for no block):
Row 1 constraints: 4
Row 2 constraints: 3
Row 3 constraints: 3,1
Row 4 constraints: 5
Row 5 constraints: 8
Row 6 constraints: 7
Row 7 constraints: 1,1,1
Row 8 constraints: 3
Row 9 constraints: 1,1,4
Row 10 constraints: 4,3
Enter column constraints (one column per line; comma-separated numbers, or empty for no block):
Column 1 constraints: 1,2,1
Column 2 constraints: 1,3,2
Column 3 constraints: 3,2,1
Column 4 constraints: 7,2
Column 5 constraints: 5
Column 6 constraints: 3,1
Column 7 constraints: 4,3
Column 8 constraints: 2,3
Column 9 constraints: 1,2
Column 10 constraints: 1
Total filled squares (expected): 50

After Step 1 (Global Confidence Filling) - Grid:
-1 -1 -1 1 -1 -1 -1 -1 -1 -1
-1 -1 -1 1 -1 -1 -1 -1 -1 -1
-1 -1 1 1 -1 -1 1 -1 -1 -1
-1 -1 -1 1 -1 -1 1 -1 -1 -1
-1 1 1 1 1 1 1 1 -1 -1
-1 -1 -1 1 1 1 1 -1 -1 -1
-1 -1 -1 1 -1 -1 -1 -1 -1 -1
-1 -1 -1 0 -1 -1 1 -1 -